In [1]:
import psycopg2
import psycopg2.extras as extras
import config_warehouse as creds
import config_lake as creds_lake
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [2]:
# Set up a connection to the warehouse
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
          +" password="+ creds.PGPASSWORD

conn = psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()

# Select table measuring points
cursor.execute((""" SELECT * FROM measuring_points """))
df_records = pd.DataFrame(cursor.fetchall())
df_measuring_points = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
#print(colnames)
df_measuring_points.columns = colnames

# Select table measuring points location
cursor.execute((""" SELECT * FROM measuring_points_coordinates """))
df_records = pd.DataFrame(cursor.fetchall())
df_measuring_points_coordinates = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_measuring_points_coordinates.columns = colnames

# Select table traffic measurement
cursor.execute((""" SELECT * FROM traffic_measurement_21 """))
df_records = pd.DataFrame(cursor.fetchall())
df_traffic_21 = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_traffic_21.columns = colnames

# Select table charging_stations_static
cursor.execute((""" SELECT * FROM charging_stations_static """))
df_records = pd.DataFrame(cursor.fetchall())
df_charging_stations_static = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_charging_stations_static.columns = colnames

# Select table charging_stations_location
cursor.execute((""" SELECT * FROM charging_stations_location """))
df_records = pd.DataFrame(cursor.fetchall())
df_charging_stations_location = pd.DataFrame(df_records)
colnames = [desc[0] for desc in cursor.description]
df_charging_stations_location.columns = colnames

conn.close()

Connected!


In [3]:
# # Overview
# print(df_measuring_points.head(),'\n',
# df_measuring_points_coordinates.head(), '\n',
# df_traffic_21.head(), '\n',
# df_charging_stations_static.head(), '\n',
# df_charging_stations_location.head())

In [4]:
#Merge dataframes # Measuring points w/ traffic
print('Lenght dataframes: ',
len(df_measuring_points),',',
len(df_measuring_points_coordinates),',',
len(df_traffic_21))
df1 = df_measuring_points.merge(df_measuring_points_coordinates, on = 'nr_measuring_point', how='left')
df_points = df_traffic_21.merge(df1, on = 'nr_measuring_point', how= 'left')
df_points.drop_duplicates(subset=['nr_measuring_point'], inplace=True)
df_points.drop(['nr_x', 'nr_y'], axis=1, inplace=True)
df_points

Lenght dataframes:  459 , 918 , 386


,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord
0,2,0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,2545269,1158463
2,5,6918,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,2724500,1206725
4,9,7865,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,2653895,1180111
6,10,0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,2686520,1163452
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,2780967,1148640
...,...,...,...,...,...,...,...,...,...
762,852,0,ECHANGEUR GD.ST.BERNARD EST (AR),in Betrieb,Online+Statistik,A 9,VS,2573706,1107161
764,853,0,BIENNE NORD (AB),in Betrieb,Online+Statistik,A 16,BE,2587263,1222983
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,2690786,1278336
768,855,27060,BENKEN S (AB),in Betrieb,nur Statistik,A 4,ZH,2691429,1277447


In [5]:
df_points.isnull().values.any()

False

In [6]:
#Merge dataframes # Chargin stations w/ location
print('Lenght dataframes: ',
len(df_charging_stations_static),',',
len(df_charging_stations_location))
df_stations = df_charging_stations_static.merge(df_charging_stations_location, on='charging_station_id',
                                                how='left')
df_stations

Lenght dataframes:  5050 , 3826


,evse_id,charging_station_id,operator_id,city,street,power,id_postal_code,coordinate_east,coordinate_nord
0,CH*EWD*E123*0033,CH*EWD*E123*0033,CH*EWD,Davos,Bahnhofstrasse,22,1146,9.845910,46.813520
1,CH*EWD*E123*0027,CH*EWD*E123*0027,CH*EWD,Davos,Bahnhofstrasse,22,1146,9.845860,46.813500
2,CH*REPE116*01*1,CH*REPE116*01*1,CH*REP,Davos,Flüalastrasse,22,1146,9.851364,46.811108
3,CH*REPE116*01*2,CH*REPE116*01*1,CH*REP,Davos,Flüalastrasse,22,1146,9.851364,46.811108
4,CH*ECUEKWZFPH3QU3DLEHRXX5LBBLPNW2,CH*ECUEKWZFPH3QU3DLEHRXX5LBBLPNW2,CH*ECU,Davos Dorf,Hotel Inn Promenade 116,11,1146,9.836806,46.802939
...,...,...,...,...,...,...,...,...,...
5045,CH*ECUESQTDWS8E7GCXWWNLXPGKTY7XKW,CH*ECUESQTDWS8E7GCXWWNLXPGKTY7XKW,CH*ECU,Zürich,Reckenholzstrasse 191,22,1563,8.516089,47.429032
5046,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECU,Domat/Ems,Via la Val 1,22,779,9.464860,46.837768
5047,CH*ECUELLVS9S9GYJTCYSNNLL8MBJSVYX,CH*ECUESNV3BFTL6Q2L682S7HZFMSR6TQ,CH*ECU,Domat/Ems,Via la Val 1,22,779,9.464860,46.837768
5048,CH*ECUETWY3BAC7LALSQ3YMWE9RPWQGHX,CH*ECUETWY3BAC7LALSQ3YMWE9RPWQGHX,CH*ECU,Luzern,Littauerboden 2,11,1508,8.264859,47.061965


In [7]:
df_stations.isnull().values.any()

False

In [8]:
# Approximate Formula to transform Swiss coordinates to WGS84 coordinates, deviation in ~1-3 meters
def lv95_to_wgs84_coordinates(y, x): # y = coordinate-east, x = coordinate-nord
    # convert into the civilian system in the unit 1000 km
    y = (y-2600000)/1000000
    x = (x-1200000)/1000000
    # Calculate longitude / latitude in the unit 10000"
    E = 2.6779094\
    + 4.728982 * y \
    + 0.791484 * y * x\
    + 0.1306 *y * x**2\
    - 0.0436 * y**3
    N = 16.9023892\
    + 3.238272 * x \
    - 0.270978 * y**2\
    - 0.002528 * x**2\
    - 0.0447 * y**2 * x\
    - 0.0140 * x**3
    #Convert longitude and latitude to unit °
    E = round(E * 100/36,6)
    N = round(N * 100/36,6)
    return [E, N]

# # Trial
# lv95_to_wgs84_coordinates(y = 2700000,
#                           x = 1100000)
# lv95_to_wgs84_coordinates(y = 2545269,
#                           x = 1158463)

In [9]:
len(df_points)
df_points

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord
0,2,0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,2545269,1158463
2,5,6918,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,2724500,1206725
4,9,7865,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,2653895,1180111
6,10,0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,2686520,1163452
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,2780967,1148640
...,...,...,...,...,...,...,...,...,...
762,852,0,ECHANGEUR GD.ST.BERNARD EST (AR),in Betrieb,Online+Statistik,A 9,VS,2573706,1107161
764,853,0,BIENNE NORD (AB),in Betrieb,Online+Statistik,A 16,BE,2587263,1222983
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,2690786,1278336
768,855,27060,BENKEN S (AB),in Betrieb,nur Statistik,A 4,ZH,2691429,1277447


In [10]:
# Transform coordinates
east_list = []
nord_list = []
for i in range(0, 386):
    east = df_points.iloc[i, 7]
    nord = df_points.iloc[i, 8]
    #print(nord, east)
    coordinate_list = lv95_to_wgs84_coordinates(y = east, x = nord)
    east_list.append(coordinate_list[0])
    nord_list.append(coordinate_list[1])
    #print(coordinate_list)
print(len(east_list), len(nord_list))
# temp_list

386 386


In [11]:
df_points['coordinate_east'] = east_list
df_points['coordinate_nord'] = nord_list
df_points

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord
0,2,0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,6.724671,46.575199
2,5,6918,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,9.075686,46.999893
4,9,7865,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,8.144237,46.769994
6,10,0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,8.568181,46.616716
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,9.794852,46.464632
...,...,...,...,...,...,...,...,...,...
762,852,0,ECHANGEUR GD.ST.BERNARD EST (AR),in Betrieb,Online+Statistik,A 9,VS,7.098525,46.115434
764,853,0,BIENNE NORD (AB),in Betrieb,Online+Statistik,A 16,BE,7.270677,47.157691
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,8.646955,47.649383
768,855,27060,BENKEN S (AB),in Betrieb,nur Statistik,A 4,ZH,8.655328,47.641300


In [12]:
# Radius formula
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculat e the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [13]:
df_radius = df_points.copy()

# Number of chargin stations within radius of 1,2,5 km
df_radius['Charging_stations_1km'] = 0
df_radius['Charging_stations_1-2km'] = 0
df_radius['Charging_stations_2-5km'] = 0
df_radius['Charging_stations_5-10km'] = 0
temp_1km_list = []
temp_2km_list = []
temp_5km_list = []
temp_10km_list = []

for row in range(0, 386):
    station_count_1km = 0
    station_count_2km = 0
    station_count_5km = 0
    station_count_10km = 0
    point_east = df_points.iloc[row, 7]
    point_nord = df_points.iloc[row, 8]
    for row_2 in range(0, 386):
        station_east = df_stations.iloc[row_2, 7]
        station_nord = df_stations.iloc[row_2, 8]
        distance = haversine(point_east, point_nord, station_east, station_nord)
        # stations within 1km
        if distance <= 1:
             station_count_1km = station_count_1km + 1
        # stations within 2km
        if distance <= 2:
             station_count_2km = station_count_2km + 1
        # stations within 5km
        if distance <= 5:
             station_count_5km = station_count_5km + 1
        # stations within 10km
        if distance <= 10:
             station_count_10km = station_count_10km + 1
        else:
            continue
    temp_1km_list.append(station_count_1km)
    temp_2km_list.append(station_count_2km)
    temp_5km_list.append(station_count_5km)
    temp_10km_list.append(station_count_10km)

df_radius['Charging_stations_1km'] = temp_1km_list
df_radius['Charging_stations_1-2km'] = [temp_2km_list[i]-temp_1km_list[i] for i in range(0, 386)]
df_radius['Charging_stations_2-5km'] = [temp_5km_list[i]-temp_2km_list[i] for i in range(0, 386)]
df_radius['Charging_stations_5-10km'] = [temp_10km_list[i]-temp_5km_list[i] for i in range(0, 386)]

In [14]:
df_radius

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord,Charging_stations_1km,Charging_stations_1-2km,Charging_stations_2-5km,Charging_stations_5-10km
0,2,0,CHALET-A-GOBET,in Betrieb,Online+Statistik,H 1,VD,6.724671,46.575199,0,0,0,0
2,5,6918,SCHWANDEN N,in Betrieb,Online+Statistik,H 17,GL,9.075686,46.999893,0,0,0,0
4,9,7865,"BRUENIG, LETZI",in Betrieb,Online+Statistik,A 8,OW,8.144237,46.769994,0,0,0,0
6,10,0,"HOSPENTAL S, ST. GOTTHARD",in Betrieb,Online+Statistik,H 2,UR,8.568181,46.616716,0,0,0,0
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,9.794852,46.464632,0,0,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,852,0,ECHANGEUR GD.ST.BERNARD EST (AR),in Betrieb,Online+Statistik,A 9,VS,7.098525,46.115434,0,1,27,0
764,853,0,BIENNE NORD (AB),in Betrieb,Online+Statistik,A 16,BE,7.270677,47.157691,0,0,0,0
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,8.646955,47.649383,0,0,0,17
768,855,27060,BENKEN S (AB),in Betrieb,nur Statistik,A 4,ZH,8.655328,47.641300,0,0,0,17


In [15]:
df_radius[df_radius['Charging_stations_1km'] != 0]

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord,Charging_stations_1km,Charging_stations_1-2km,Charging_stations_2-5km,Charging_stations_5-10km
24,25,47646,BAD RAGAZ (AB),in Betrieb,Online+Statistik,A 13,SG,9.505120,47.014617,10,14,3,0
142,123,26963,SCHAFFHAUSEN N (AB),in Betrieb,Online+Statistik,A 4,SH,8.649517,47.713819,15,2,0,0
276,225,61082,"EGERKINGEN, VERZWEIGUNG A2 (AB)",in Betrieb,Online+Statistik,A 2,SO,7.806052,47.315939,8,10,0,26
278,226,0,CRISSIER (AR),in Betrieb,nur Statistik,A 1,VD,6.562186,46.558536,16,3,0,0
362,286,130700,WALLISELLEN (AB),in Betrieb,Online+Statistik,A 1,ZH,8.616875,47.410322,9,0,0,0
368,290,102986,OFTRINGEN / ROTHRIST (AB) - AG1412,in Betrieb,Online+Statistik,A 1,AG,7.903472,47.309490,5,2,19,18
428,328,26417,ESTAVAYER-LE-LAC E (AR),in Betrieb,Online+Statistik,A 1,FR,6.871261,46.831485,14,7,3,0
646,690,16413,FLUEELEN (AB),in Betrieb,nur Statistik,A 4,UR,8.620923,46.894127,1,0,6,0
708,769,0,HERBLINGEN,Zählstelle geplant,nur Statistik,A 4,SH,8.668375,47.722490,3,14,0,0
752,841,23192,BUOCHS OST (AB),in Betrieb,nur Statistik,A 2,NW,8.452472,46.970121,7,1,0,0


In [16]:
df_radius[df_radius['Charging_stations_1-2km'] != 0]

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord,Charging_stations_1km,Charging_stations_1-2km,Charging_stations_2-5km,Charging_stations_5-10km
24,25,47646,BAD RAGAZ (AB),in Betrieb,Online+Statistik,A 13,SG,9.505120,47.014617,10,14,3,0
82,73,54272,HAERKINGEN (AB),in Betrieb,Online+Statistik,A 1,SO,7.807574,47.312220,0,18,0,26
126,114,0,BRUETTISELLEN N (AB),in Betrieb,Online+Statistik,A 1,ZH,8.642871,47.423124,0,1,8,0
142,123,26963,SCHAFFHAUSEN N (AB),in Betrieb,Online+Statistik,A 4,SH,8.649517,47.713819,15,2,0,0
174,149,0,MEX (AR),in Betrieb,Online+Statistik,A 1,VD,6.557570,46.570510,0,16,3,0
192,165,0,MARTIGNY N (AR),ausser Betrieb (Bauarbeiten),Online+Statistik,A 9,VS,7.069411,46.118770,0,27,1,0
228,194,84018,URDORF (AB),in Betrieb,Online+Statistik,A 3,ZH,8.414773,47.390599,0,5,28,0
276,225,61082,"EGERKINGEN, VERZWEIGUNG A2 (AB)",in Betrieb,Online+Statistik,A 2,SO,7.806052,47.315939,8,10,0,26
278,226,0,CRISSIER (AR),in Betrieb,nur Statistik,A 1,VD,6.562186,46.558536,16,3,0,0
284,229,72543,OFTRINGEN E (AB),in Betrieb,Online+Statistik,A 1,AG,7.948910,47.317609,0,19,7,0


In [17]:
df_radius[df_radius['Charging_stations_2-5km'] != 0]

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord,Charging_stations_1km,Charging_stations_1-2km,Charging_stations_2-5km,Charging_stations_5-10km
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,9.794852,46.464632,0,0,12,16
24,25,47646,BAD RAGAZ (AB),in Betrieb,Online+Statistik,A 13,SG,9.505120,47.014617,10,14,3,0
36,35,0,TRUEBBACH S (AB),in Betrieb,Online+Statistik,A 13,SG,9.462862,47.038667,0,0,10,17
38,38,15279,STARRKIRCH,in Betrieb,Online+Statistik,H 5,SO,7.922704,47.352135,0,0,26,10
46,43,0,PREVERENGES (AR),in Betrieb,Online+Statistik,A 1,VD,6.541364,46.529883,0,0,19,0
66,60,91017,GUNZGEN (AB),in Betrieb,Online+Statistik,A 1,SO,7.820756,47.311664,0,0,18,26
70,66,35187,SCHLIEREN (AB),in Betrieb,Online+Statistik,A 1,ZH,8.459347,47.405271,0,0,29,4
92,81,122161,"MUTTENZ, HARD (AB)",in Betrieb,Online+Statistik,A 2,BL,7.644802,47.538702,0,0,23,0
116,104,0,MURI BE S (AB),ausser Betrieb (Bauarbeiten),Online+Statistik,A 6,BE,7.506972,46.920729,0,0,16,0
126,114,0,BRUETTISELLEN N (AB),in Betrieb,Online+Statistik,A 1,ZH,8.642871,47.423124,0,1,8,0


In [18]:
df_radius[df_radius['Charging_stations_5-10km'] != 0]

,nr_measuring_point,annual_average,point_name,status,type_point,road,canton_abbreviation,coordinate_east,coordinate_nord,Charging_stations_1km,Charging_stations_1-2km,Charging_stations_2-5km,Charging_stations_5-10km
8,14,3434,"JULIER, LA VEDUTA",in Betrieb,Online+Statistik,H 3,GR,9.794852,46.464632,0,0,12,16
16,20,102196,"UMF. ZUERICH N, AFFOLTERN (AB)",ausser Betrieb (Bauarbeiten),Online+Statistik,A 1C,ZH,8.541565,47.433075,0,0,0,9
32,32,82191,DEITINGEN (AB),in Betrieb,Online+Statistik,A 1,SO,7.612048,47.224388,0,0,0,14
34,33,53412,EPTINGEN N (AB),in Betrieb,Online+Statistik,A 2,BL,7.813893,47.396459,0,0,0,18
36,35,0,TRUEBBACH S (AB),in Betrieb,Online+Statistik,A 13,SG,9.462862,47.038667,0,0,10,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,818,46404,SOLOTHURN,Zählstelle geplant,Online+Statistik,A 5,SO,7.538926,47.194936,0,0,2,12
738,828,40721,TRUEBBACH N (AB),in Betrieb,Online+Statistik,A 13,SG,9.501166,47.080807,0,0,0,27
748,839,39497,STANS NORD (AB),in Betrieb,nur Statistik,A 2,NW,8.366854,46.966349,0,0,0,8
766,854,27123,BENKEN N (AB),in Betrieb,nur Statistik,A 4,ZH,8.646955,47.649383,0,0,0,17


In [19]:
# df_final = df_radius[(df_radius['annual_average'] != 0)]
df_final = df_radius[df_radius['annual_average'] != 0].groupby(['road']).agg({'annual_average': 'sum',
                                 'Charging_stations_1km' : 'sum',
                                 'Charging_stations_1-2km' : 'sum',
                                 'Charging_stations_2-5km' : 'sum',
                                 'Charging_stations_5-10km' : 'sum'})
df_final.reset_index(inplace=True)
df_final.columns = ['National_road', 'Monthly_traffic_average', 'Charging_stations_1km',
                     'Charging_stations_1_to_2km', 'Charging_stations_2_to_5km', 'Charging_stations_5_to_10km']
df_final.sort_values(by = 'Monthly_traffic_average', ascending = False, inplace = True)
df_final

,National_road,Monthly_traffic_average,Charging_stations_1km,Charging_stations_1_to_2km,Charging_stations_2_to_5km,Charging_stations_5_to_10km
0,A 1,2517734,28,79,102,137
12,A 2,1189065,15,11,51,114
17,A 3,1100886,0,5,29,292
18,A 4,675519,16,2,42,105
20,A 5,565590,0,0,16,39
...,...,...,...,...,...,...
61,H 509,3927,0,0,8,0
62,H 560,2939,0,0,0,0
63,H 566,2244,0,0,0,0
58,H 416,2022,0,0,0,0


In [20]:
df_import = df_final[df_final['Charging_stations_2_to_5km'] != 0]
df_import

,National_road,Monthly_traffic_average,Charging_stations_1km,Charging_stations_1_to_2km,Charging_stations_2_to_5km,Charging_stations_5_to_10km
0,A 1,2517734,28,79,102,137
12,A 2,1189065,15,11,51,114
17,A 3,1100886,0,5,29,292
18,A 4,675519,16,2,42,105
20,A 5,565590,0,0,16,39
24,A 9,502156,0,1,35,19
3,A 13,495972,10,14,3,50
21,A 6,169029,0,0,16,0
1,A 11,86338,0,0,9,0
30,H 13,81403,0,6,17,6


In [21]:
# Inserting dataframe into database
def execute_values(conn, df, table):
    # Set up a connection to the warehouse
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
              +" password="+ creds.PGPASSWORD

    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()

    # Set up data
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cur = conn.cursor()
    try:
        extras.execute_values(cur, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return
    print("the dataframe is inserted")
    cur.close()

In [22]:
# Creating tables
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE IF NOT EXISTS National_roads_with_charging_station_nearby (
            ID SERIAL PRIMARY KEY,
            National_road VARCHAR(255) NOT NULL,
            Monthly_traffic_average INT NOT NULL,
            Charging_stations_1km INT NOT NULL,
            Charging_stations_1_to_2km INT NOT NULL,
            Charging_stations_2_to_5km INT NOT NULL,
            Charging_stations_5_to_10km INT NOT NULL
            )
        """,
        """
        CREATE TABLE IF NOT EXISTS National_roads_with_charging_station_in_2_to_5km_radius (
            ID SERIAL PRIMARY KEY,
            National_road VARCHAR(255) NOT NULL,
            Monthly_traffic_average INT NOT NULL,
            Charging_stations_1km INT NOT NULL,
            Charging_stations_1_to_2km INT NOT NULL,
            Charging_stations_2_to_5km INT NOT NULL,
            Charging_stations_5_to_10km INT NOT NULL
            )
        """)
    try:
        # Set up a connection to the postgres server.
        conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD

        conn = psycopg2.connect(conn_string)
        print("Connected!")

        # Create a cursor object
        cursor = conn.cursor()

        # Create tables
        for command in commands:
            cursor.execute(command)
        print('Tables created')

        # close communication with the PostgreSQL database server
        cursor.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

create_tables()

Connected!
Tables created


In [23]:
# Import data from dataframe
execute_values(conn, df_final, 'National_roads_with_charging_station_nearby')
execute_values(conn, df_import, 'National_roads_with_charging_station_in_2_to_5km_radius')

Connected!
the dataframe is inserted
Connected!
the dataframe is inserted
